<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install scikit-learn # for metrics
!pip install --upgrade kagglehub

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [3]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import openai
import pandas as pd
import numpy as np
import os
import kagglehub

from google.colab import userdata

# print(openai.__version__)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_NLP')

In [4]:
huggingface_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = huggingface_token

In [5]:
from datasets import load_dataset

In [6]:
# function to read multiple csv files from directory and combine into one
def read_multi_csv(path):
  csv_files = []
  for filename in os.listdir(path):
      if filename.endswith(".csv"):
          csv_files.append(filename)

  # read csv files and join into one
  return pd.concat([pd.read_csv(os.path.join(path, filename)) for filename in csv_files])

In [7]:
# function to read single csv file from directory
def read_single_csv(path):
  for filename in os.listdir(path):
      if filename.endswith(".csv"):
          csv_file_path = os.path.join(path, filename)
          break  # Stop after finding the first CSV file

  return pd.read_csv(csv_file_path)

## SA1 TCDSAD


In [ ]:
# load first dataset (SENTIMENT ANALYSIS)
path = kagglehub.dataset_download("jsonali2003/text-classification-dataset-for-sentiment-analysis")
data_sa1 = read_multi_csv(path)
data_sa1.head()

100%|██████████| 5.42k/5.42k [00:00<00:00, 988kB/s]

Extracting files...


,sentence,sentiment
0,Social justice requires addressing systemic in...,positive
1,Racial discrimination in policing and criminal...,negative
2,Equal access to education is essential for bre...,positive
3,Gender-based violence and discrimination are p...,negative
4,"Immigrant rights are human rights, and policie...",positive


In [ ]:
# count sentiment split
data_sa1['sentiment'].value_counts()


,count
sentiment,
positive,66
negative,34


In [ ]:
data_sa1.rename(columns={'sentence': 'text'}, inplace=True)
# add column for binary label
data_sa1['label'] = data_sa1['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
data_sa1.head()

,text,sentiment,label
0,Social justice requires addressing systemic in...,positive,1
1,Racial discrimination in policing and criminal...,negative,0
2,Equal access to education is essential for bre...,positive,1
3,Gender-based violence and discrimination are p...,negative,0
4,"Immigrant rights are human rights, and policie...",positive,1


In [ ]:
# save to google drive
data_sa1.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_sa1.csv', index=False)

## SA2 SAED (was removed from kaggle ??)


In [ ]:
# social media posts (SENTIMENT ANALYSIS)
path = kagglehub.dataset_download("prishasawhney/sentiment-analysis-evaluation-dataset")

data_sa2 = read_multi_csv(path)
data_sa2.head()

KaggleApiHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/prishasawhney/sentiment-analysis-evaluation-dataset. The server reported the following issues: Permission 'datasets.get' was denied
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

In [ ]:
data_sa2.rename(columns={'Text': 'text', 'Label': 'sentiment'}, inplace=True)
# add column for binary label
data_sa2['label'] = data_sa2['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)


In [ ]:
data_sa2.head()

,text,sentiment,label
0,The government's recent policies have received...,positive,1
1,Political analysts are divided on the long-ter...,negative,0
2,Efforts to promote unity among political facti...,positive,1
3,"Despite allegations of corruption, the governm...",negative,0
4,The recent diplomatic initiatives have been me...,positive,1


In [ ]:
data_sa2['sentiment'].value_counts()

,count
sentiment,
positive,113
negative,96


In [ ]:
# save to google drive
data_sa2.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data_sa2.csv', index=False)

## TC1 IMDB

In [ ]:
# IMDB genre texts (TEXT CLASSIFICATION)
path = kagglehub.dataset_download("zulkarnainsaurav/imdb-text-classification")


data_tc = read_single_csv(path)
data_tc.head()


100%|██████████| 748k/748k [00:00<00:00, 90.4MB/s]

Extracting files...


,movie_id,description,genre
0,tt12783454,Elle Evans (Joey King) has finally completed h...,romance
1,tt1798632,A young girl tries to understand how she myste...,horror
2,tt9214832,"In 1800s England, a well meaning but selfish y...",comedy
3,tt8522006,Abby Holland (Kristen Stewart) and Harper Cald...,romance
4,tt21249656,Olga and Maks are 15 years apart. She is a suc...,romance


In [ ]:
# genre breakdown
data_tc['genre'].value_counts()

,count
genre,
romance,250
horror,250
comedy,250
action,250


In [ ]:
# add label column romance = 0, horror = 1, comedy = 2, action = 3
data_tc['label'] = data_tc['genre'].apply(lambda x: 0 if x == 'romance' else 1 if x == 'horror' else 2 if x == 'comedy' else 3)
data_tc.rename(columns={'description': 'text'}, inplace=True)
data_tc.head()


,movie_id,text,genre,label
0,tt12783454,Elle Evans (Joey King) has finally completed h...,romance,0
1,tt1798632,A young girl tries to understand how she myste...,horror,1
2,tt9214832,"In 1800s England, a well meaning but selfish y...",comedy,2
3,tt8522006,Abby Holland (Kristen Stewart) and Harper Cald...,romance,0
4,tt21249656,Olga and Maks are 15 years apart. She is a suc...,romance,0


In [ ]:
data_tc['label'].value_counts()

,count
label,
0,250
1,250
2,250
3,250


In [ ]:
# save dataframe to google drive
# save to drive
data_tc.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_tc1.csv', index=False)


## TC2 SPAM


In [ ]:
# SMS SPAM (TEXT CLASSIFICATION)
path = kagglehub.dataset_download("ashfakyeafi/spam-email-classification")


data_tc2 = read_single_csv(path)
data_tc2.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# add binary label column (spam = 1)
data_tc2['label'] = data_tc2['Category'].apply(lambda x: 1 if x == 'spam' else 0)

# rename Message column
data_tc2.rename(columns={'Message': 'text'}, inplace=True)
data_tc2.head()

,Category,text,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
# class split
data_tc2['label'].value_counts()

,count
label,
0,4826
1,747


In [ ]:
# keep only ham and spam
#data_tc2 = data_tc2[data_tc2['label'].isin(['ham', 'spam'])]
#data_tc2.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# save dataframe to google drive
# save to drive
data_tc2.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_tc2.csv', index=False)


## tran1

In [8]:
# greek to english dataset (TRANSLATION)

path = "/content/drive/MyDrive/ell.txt"

# txt file to dataframe
data_greek = pd.read_csv(path, sep='\t', header=None)
data_greek.head()

,0,1,2
0,Go.,Πάμε.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Πήγαινε.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Run!,Τρέξε!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,Τρέξτε!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run.,Τρέχα.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [9]:
# rename columns
data_greek.rename(columns={0: 'english', 1: 'greek'}, inplace=True)
# remove column
data_greek = data_greek.drop(columns=[2])
data_greek.head()


,english,greek
0,Go.,Πάμε.
1,Go.,Πήγαινε.
2,Run!,Τρέξε!
3,Run!,Τρέξτε!
4,Run.,Τρέχα.


In [10]:
len(data_greek)

17549

In [14]:
# shuffle data and take 200 samples
data_greek = data_greek.sample(frac=1).reset_index(drop=True)
data_greek = data_greek.head(200)
data_greek.head()

,english,greek
0,You must clear the table.,Πρέπει να καθαρίσεις το τραπέζι.
1,He came to Japan when he was 10 years old.,"Ήρθε στην Ιαπωνία, όταν ήταν 10 ετών."
2,Your friends are waiting for you.,Οι φίλοι σας σας περιμένουν.
3,Tom wants to be a lawyer.,Ο Τομ θέλει να γίνει δικηγόρος.
4,Are you ready for spring?,Είσαι έτοιμος για την άνοιξη?


In [13]:
len(data_greek)

400

In [15]:
# save to drive
data_greek.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_eng_greek.csv', index=False)

## summarization


In [ ]:
# load dataset (SUMMARIZATION)
path =

In [ ]:
# third dataset for sentiment analysis
path = kagglehub.dataset_download("kashishparmar02/social-media-sentiments-analysis-dataset")
data_sa3 = read_single_csv(path)
data_sa3.head()


,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [ ]:

data_sa3.rename(columns={'Text': 'text', 'Sentiment': 'sentiment'}, inplace=True)
# remove unnecessary columns
data_sa3 = data_sa3[['text', 'sentiment', 'label', 'Platform']]

# add column for binary label
data_sa3['label'] = data_sa3['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
# shuffle data
data_sa3 = data_sa3.sample(frac=1).reset_index(drop=True)
data_sa3.head()

,text,sentiment,label,Platform
0,Pride in accomplishing personal and professio...,Pride,0,Instagram
1,Excitement building up for a surprise birthda...,Excitement,0,Facebook
2,"Melancholy as a companion, painting the canva...",Melancholy,0,Facebook
3,Attending a workshop on time management to enh...,Neutral,0,Twitter
4,Attending a live music concert tonight. ...,Positive,0,Instagram


In [ ]:
data_sa3.value_counts('sentiment')

,count
sentiment,
Positive,44
Joy,42
Excitement,32
Happy,14
Contentment,14
...,...
InnerJourney,1
Blessed,1
Inspiration,1
